In [1]:
import logging
import os.path
import time
from collections import OrderedDict
import sys

import numpy as np
import torch.nn.functional as F
from torch import optim

from braindecode.models.deep4 import Deep4Net
from braindecode.datasets.bcic_iv_2a import BCICompetition4Set2A
from braindecode.experiments.experiment import Experiment
from braindecode.experiments.monitors import LossMonitor, MisclassMonitor, \
    RuntimeMonitor
from braindecode.experiments.stopcriteria import MaxEpochs, NoDecrease, Or
from braindecode.datautil.iterators import BalancedBatchSizeIterator
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from braindecode.datautil.splitters import split_into_two_sets
from braindecode.torch_ext.constraints import MaxNormDefaultConstraint
from braindecode.torch_ext.util import set_random_seeds, np_to_var
from braindecode.mne_ext.signalproc import mne_apply
from braindecode.datautil.signalproc import (bandpass_cnt,
                                             exponential_running_standardize)
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne

log = logging.getLogger(__name__)

<p>Here's the description from the paper</p>
<img src="DeepConvNet.png" style="width: 700px; float:left;">

<p>Here's the description from the paper</p>
<img src="DeepConvNetDetail.png" style="width: 700px; float:left;">

In [2]:
def run_exp(data_folder, subject_id, low_cut_hz, model, cuda):
    ival = [-500, 4000]
    max_epochs = 1600
    max_increase_epochs = 160
    batch_size = 60
    high_cut_hz = 38
    factor_new = 1e-3
    init_block_size = 1000
    valid_set_fraction = 0.2

    train_filename = 'A{:02d}T.gdf'.format(subject_id)
    test_filename = 'A{:02d}E.gdf'.format(subject_id)
    train_filepath = os.path.join(data_folder, train_filename)
    test_filepath = os.path.join(data_folder, test_filename)
    train_label_filepath = train_filepath.replace('.gdf', '.mat')
    test_label_filepath = test_filepath.replace('.gdf', '.mat')

    train_loader = BCICompetition4Set2A(
        train_filepath, labels_filename=train_label_filepath)
    test_loader = BCICompetition4Set2A(
        test_filepath, labels_filename=test_label_filepath)
    train_cnt = train_loader.load()
    test_cnt = test_loader.load()

    # Preprocessing

    train_cnt = train_cnt.drop_channels(['STI 014', 'EOG-left',
                                         'EOG-central', 'EOG-right'])
    assert len(train_cnt.ch_names) == 22
    # lets convert to millvolt for numerical stability of next operations
    train_cnt = mne_apply(lambda a: a * 1e6, train_cnt)
    train_cnt = mne_apply(
        lambda a: bandpass_cnt(a, low_cut_hz, high_cut_hz, train_cnt.info['sfreq'],
                               filt_order=3,
                               axis=1), train_cnt)
    train_cnt = mne_apply(
        lambda a: exponential_running_standardize(a.T, factor_new=factor_new,
                                                  init_block_size=init_block_size,
                                                  eps=1e-4).T,
        train_cnt)

    test_cnt = test_cnt.drop_channels(['STI 014', 'EOG-left',
                                       'EOG-central', 'EOG-right'])
    assert len(test_cnt.ch_names) == 22
    test_cnt = mne_apply(lambda a: a * 1e6, test_cnt)
    test_cnt = mne_apply(
        lambda a: bandpass_cnt(a, low_cut_hz, high_cut_hz, test_cnt.info['sfreq'],
                               filt_order=3,
                               axis=1), test_cnt)
    test_cnt = mne_apply(
        lambda a: exponential_running_standardize(a.T, factor_new=factor_new,
                                                  init_block_size=init_block_size,
                                                  eps=1e-4).T,
        test_cnt)

    marker_def = OrderedDict([('Left Hand', [1]), ('Right Hand', [2],),
                              ('Foot', [3]), ('Tongue', [4])])

    train_set = create_signal_target_from_raw_mne(train_cnt, marker_def, ival)
    test_set = create_signal_target_from_raw_mne(test_cnt, marker_def, ival)

    train_set, valid_set = split_into_two_sets(
        train_set, first_set_fraction=1-valid_set_fraction)

    set_random_seeds(seed=20190706, cuda=cuda)

    n_classes = 4
    n_chans = int(train_set.X.shape[1])
    input_time_length = train_set.X.shape[2]
    if model == 'shallow':
        model = ShallowFBCSPNet(n_chans, n_classes, input_time_length=input_time_length,
                            final_conv_length='auto').create_network()
    elif model == 'deep':
        model = Deep4Net(n_chans, n_classes, input_time_length=input_time_length,
                            final_conv_length='auto').create_network()
    if cuda:
        model.cuda()
    log.info("Model: \n{:s}".format(str(model)))

    optimizer = optim.Adam(model.parameters())

    iterator = BalancedBatchSizeIterator(batch_size=batch_size)

    stop_criterion = Or([MaxEpochs(max_epochs),
                         NoDecrease('valid_misclass', max_increase_epochs)])

    monitors = [LossMonitor(), MisclassMonitor(), RuntimeMonitor()]

    model_constraint = MaxNormDefaultConstraint()

    exp = Experiment(model, train_set, valid_set, test_set, iterator=iterator,
                     loss_function=F.nll_loss, optimizer=optimizer,
                     model_constraint=model_constraint,
                     monitors=monitors,
                     stop_criterion=stop_criterion,
                     remember_best_column='valid_misclass',
                     run_after_early_stop=True, cuda=cuda)
    exp.run()
    return exp

In [3]:
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                    level=logging.DEBUG, stream=sys.stdout)
# Should contain both .gdf files and .mat-labelfiles from competition
data_folder = './data/BCICIV_2a_gdf/'
subject_id = 1 # 1-9
low_cut_hz = 4 # 0 or 4
model = 'deep' #'shallow' or 'deep'
cuda = False
exp = run_exp(data_folder, subject_id, low_cut_hz, model, cuda)
log.info("Last 10 epochs")
log.info("\n" + str(exp.epochs_df.iloc[-10:]))

Extracting EDF parameters from ./data/BCICIV_2a_gdf/A01T.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


/home/canonyeee/anaconda3/lib/python3.6/site-packages/braindecode/datasets/bcic_iv_2a.py:20: RuntimeWarning: Overlapping events detected. Use find_edf_events for the original events.
  raw_edf = mne.io.read_raw_edf(self.filename, stim_channel='auto')
/home/canonyeee/anaconda3/lib/python3.6/site-packages/braindecode/datasets/bcic_iv_2a.py:20: RuntimeWarning: Interpolating stim channel. Events may jitter.
  raw_edf = mne.io.read_raw_edf(self.filename, stim_channel='auto')
/home/canonyeee/anaconda3/lib/python3.6/site-packages/braindecode/datasets/bcic_iv_2a.py:20: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw_edf = mne.io.read_raw_edf(self.filename, stim_channel='auto')


Extracting EDF parameters from ./data/BCICIV_2a_gdf/A01E.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 686999  =      0.000 ...  2747.996 secs...


/home/canonyeee/anaconda3/lib/python3.6/site-packages/braindecode/datasets/bcic_iv_2a.py:20: RuntimeWarning: Overlapping events detected. Use find_edf_events for the original events.
  raw_edf = mne.io.read_raw_edf(self.filename, stim_channel='auto')
/home/canonyeee/anaconda3/lib/python3.6/site-packages/braindecode/datasets/bcic_iv_2a.py:20: RuntimeWarning: Interpolating stim channel. Events may jitter.
  raw_edf = mne.io.read_raw_edf(self.filename, stim_channel='auto')
/home/canonyeee/anaconda3/lib/python3.6/site-packages/braindecode/datasets/bcic_iv_2a.py:20: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw_edf = mne.io.read_raw_edf(self.filename, stim_channel='auto')


2019-02-13 06:48:48,526 INFO : Trial per class:
Counter({'Tongue': 72, 'Foot': 72, 'Right Hand': 72, 'Left Hand': 72})
2019-02-13 06:48:48,716 INFO : Trial per class:
Counter({'Left Hand': 72, 'Right Hand': 72, 'Foot': 72, 'Tongue': 72})
2019-02-13 06:48:48,830 INFO : Model: 
Sequential(
  (dimshuffle): Expression(expression=_transpose_time_to_spat)
  (conv_time): Conv2d(1, 25, kernel_size=(10, 1), stride=(1, 1))
  (conv_spat): Conv2d(25, 25, kernel_size=(1, 22), stride=(1, 1), bias=False)
  (bnorm): BatchNorm2d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_nonlin): Expression(expression=elu)
  (pool): MaxPool2d(kernel_size=(3, 1), stride=(3, 1), padding=0, dilation=1, ceil_mode=False)
  (pool_nonlin): Expression(expression=identity)
  (drop_2): Dropout(p=0.5)
  (conv_2): Conv2d(25, 50, kernel_size=(10, 1), stride=(1, 1), bias=False)
  (bnorm_2): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (nonlin_2): Expression(expres

...

2019-02-13 07:34:07,265 INFO : valid_loss                0.12670
2019-02-13 07:34:07,266 INFO : test_loss                 0.97984
2019-02-13 07:34:07,267 INFO : train_misclass            0.01389
2019-02-13 07:34:07,268 INFO : valid_misclass            0.05172
2019-02-13 07:34:07,268 INFO : test_misclass             0.36111
2019-02-13 07:34:07,269 INFO : runtime                   6.15661
2019-02-13 07:34:07,270 INFO : 
2019-02-13 07:34:10,474 INFO : Time only for training updates: 3.20s
2019-02-13 07:34:13,433 INFO : Epoch 347
2019-02-13 07:34:13,434 INFO : train_loss                0.03796
2019-02-13 07:34:13,435 INFO : valid_loss                0.11187
2019-02-13 07:34:13,436 INFO : test_loss                 0.91330
2019-02-13 07:34:13,437 INFO : train_misclass            0.01042
2019-02-13 07:34:13,438 INFO : valid_misclass            0.05172
2019-02-13 07:34:13,439 INFO : test_misclass             0.35069
2019-02-13 07:34:13,440 INFO : runtime                   6.19557
2019-02-13 07

2019-02-13 07:35:27,585 INFO : valid_misclass            0.01724
2019-02-13 07:35:27,585 INFO : test_misclass             0.35069
2019-02-13 07:35:27,586 INFO : runtime                   6.28575
2019-02-13 07:35:27,587 INFO : 
2019-02-13 07:35:30,862 INFO : Time only for training updates: 3.27s
2019-02-13 07:35:33,808 INFO : Epoch 360
2019-02-13 07:35:33,809 INFO : train_loss                0.03098
2019-02-13 07:35:33,810 INFO : valid_loss                0.06019
2019-02-13 07:35:33,811 INFO : test_loss                 1.00341
2019-02-13 07:35:33,812 INFO : train_misclass            0.00347
2019-02-13 07:35:33,813 INFO : valid_misclass            0.00000
2019-02-13 07:35:33,813 INFO : test_misclass             0.32986
2019-02-13 07:35:33,814 INFO : runtime                   6.63244
2019-02-13 07:35:33,815 INFO : 
2019-02-13 07:35:33,820 INFO : New best valid_misclass: 0.000000
2019-02-13 07:35:33,821 INFO : 
2019-02-13 07:35:37,132 INFO : Time only for training updates: 3.31s
2019-02-13

2019-02-13 07:36:42,411 INFO : valid_misclass            0.00000
2019-02-13 07:36:42,412 INFO : test_misclass             0.35417
2019-02-13 07:36:42,413 INFO : runtime                   6.10888
2019-02-13 07:36:42,413 INFO : 
2019-02-13 07:36:42,417 INFO : New best valid_misclass: 0.000000
2019-02-13 07:36:42,418 INFO : 
2019-02-13 07:36:45,557 INFO : Time only for training updates: 3.14s
2019-02-13 07:36:48,521 INFO : Epoch 372
2019-02-13 07:36:48,523 INFO : train_loss                0.01300
2019-02-13 07:36:48,523 INFO : valid_loss                0.03324
2019-02-13 07:36:48,524 INFO : test_loss                 0.99022
2019-02-13 07:36:48,525 INFO : train_misclass            0.00000
2019-02-13 07:36:48,526 INFO : valid_misclass            0.00000
2019-02-13 07:36:48,527 INFO : test_misclass             0.33333
2019-02-13 07:36:48,527 INFO : runtime                   6.12258
2019-02-13 07:36:48,528 INFO : 
2019-02-13 07:36:48,532 INFO : New best valid_misclass: 0.000000
2019-02-13 07:

2019-02-13 07:37:57,768 INFO : test_loss                 0.94039
2019-02-13 07:37:57,769 INFO : train_misclass            0.00000
2019-02-13 07:37:57,770 INFO : valid_misclass            0.00000
2019-02-13 07:37:57,771 INFO : test_misclass             0.32639
2019-02-13 07:37:57,772 INFO : runtime                   6.55459
2019-02-13 07:37:57,772 INFO : 
2019-02-13 07:37:57,776 INFO : New best valid_misclass: 0.000000
2019-02-13 07:37:57,778 INFO : 
2019-02-13 07:38:01,046 INFO : Time only for training updates: 3.27s
2019-02-13 07:38:04,020 INFO : Epoch 384
2019-02-13 07:38:04,021 INFO : train_loss                0.01055
2019-02-13 07:38:04,022 INFO : valid_loss                0.02133
2019-02-13 07:38:04,023 INFO : test_loss                 0.93731
2019-02-13 07:38:04,024 INFO : train_misclass            0.00000
2019-02-13 07:38:04,025 INFO : valid_misclass            0.00000
2019-02-13 07:38:04,025 INFO : test_misclass             0.33681
2019-02-13 07:38:04,026 INFO : runtime        

2019-02-13 07:39:12,456 INFO : train_loss                0.00738
2019-02-13 07:39:12,457 INFO : valid_loss                0.01652
2019-02-13 07:39:12,458 INFO : test_loss                 0.91253
2019-02-13 07:39:12,459 INFO : train_misclass            0.00000
2019-02-13 07:39:12,460 INFO : valid_misclass            0.00000
2019-02-13 07:39:12,460 INFO : test_misclass             0.31944
2019-02-13 07:39:12,461 INFO : runtime                   6.07998
2019-02-13 07:39:12,462 INFO : 
2019-02-13 07:39:12,466 INFO : New best valid_misclass: 0.000000
2019-02-13 07:39:12,467 INFO : 
2019-02-13 07:39:15,635 INFO : Time only for training updates: 3.17s
2019-02-13 07:39:18,607 INFO : Epoch 396
2019-02-13 07:39:18,609 INFO : train_loss                0.00993
2019-02-13 07:39:18,612 INFO : valid_loss                0.01814
2019-02-13 07:39:18,613 INFO : test_loss                 0.94219
2019-02-13 07:39:18,615 INFO : train_misclass            0.00000
2019-02-13 07:39:18,616 INFO : valid_misclass 

In [4]:
data_folder = './data/BCICIV_2a_gdf/'
train_filename = 'A{:02d}T.gdf'.format(subject_id)
test_filename = 'A{:02d}E.gdf'.format(subject_id)
train_filepath = os.path.join(data_folder, train_filename)
test_filepath = os.path.join(data_folder, test_filename)
train_label_filepath = train_filepath.replace('.gdf', '.mat')
test_label_filepath = test_filepath.replace('.gdf', '.mat')
train_loader = BCICompetition4Set2A(
        train_filepath, labels_filename=train_label_filepath)

# cnt = self.extract_data()
# events, artifact_trial_mask = self.extract_events(cnt)
# cnt.info['events'] = events
# cnt.info['artifact_trial_mask'] = artifact_trial_mask



In [5]:
Stop
import mne
raw_edf = mne.io.read_raw_edf(train_filepath, stim_channel='auto')
raw_edf.load_data()
data = raw_edf.get_data()

NameError: name 'Stop' is not defined

In [ ]:
raw_edf.info

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots()
ax.plot(exp.epochs_df['test_misclass'])


In [1]:
exp.epochs_df.save()

NameError: name 'exp' is not defined

In [ ]:
exp.epochs_df.to_pickle('DeepConvNetFinalRsl')